<a href="https://colab.research.google.com/github/charan028/tidal-hack/blob/main/tidal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install earthengine-api
import ee
ee.Authenticate()



EEException: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.

In [2]:
ee.Initialize(project='tidal-hack25tex-230')


In [21]:
region = ee.Geometry.Rectangle([-125, 24, -66, 49]) # California
sample_points = ee.FeatureCollection.randomPoints(region, 200)

start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 12, 31)
all_rows = []

while start_date <= end_date:
    next_month = start_date + timedelta(days=30)
    s_str = start_date.strftime('%Y-%m-%d')
    e_str = next_month.strftime('%Y-%m-%d')
    print(f"📅 {s_str} to {e_str}")

    try:
        burned = ee.ImageCollection("MODIS/061/MCD64A1").filterDate(s_str, e_str).select('BurnDate').mean().rename('BurnedArea')
        ndvi = ee.ImageCollection("COPERNICUS/S2_SR") \
            .filterDate(s_str, e_str).filterBounds(region) \
            .map(lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI')) \
            .mean()
        climate = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H") \
            .filterDate(s_str, e_str).select(['Tair_f_inst', 'Wind_f_inst', 'Qair_f_inst']) \
            .mean().rename(['Temperature', 'WindSpeed', 'Humidity'])
        elevation = ee.Image("USGS/SRTMGL1_003").select('elevation')

        composite = ee.Image.cat([burned, ndvi, climate, elevation])
        sampled = composite.sampleRegions(collection=sample_points, scale=30, geometries=True)
        data = sampled.getInfo()

        for f in data['features']:
            p = f['properties']
            all_rows.append({
                'date': s_str,
                'NDVI': p.get('NDVI'),
                'Temperature': p.get('Temperature'),
                'Humidity': p.get('Humidity'),
                'WindSpeed': p.get('WindSpeed'),
                'Elevation': p.get('elevation'),
                'BurnedArea': p.get('BurnedArea', 0)
            })

    except Exception as e:
        print(f"⚠️ Skipped {s_str}: {e}")

    start_date = next_month

# Convert to DataFrame
df = pd.DataFrame(all_rows)

# Classify risk level
def classify_risk(val):
    if val is None:
        return 'Unknown'
    elif val > 150:
        return 'High'
    elif val > 50:
        return 'Medium'
    elif val > 0:
        return 'Low'
    else:
        return 'No Fire'

df['risk_level'] = df['BurnedArea'].fillna(0).apply(classify_risk)
df.drop(columns=['BurnedArea'], inplace=True)

# Save CSV
df.to_csv("wildfire_data_2020_2024.csv", index=False)
print("✅ Saved as wildfire_data_2020_2024.csv")

📅 2020-01-01 to 2020-01-31
📅 2020-01-31 to 2020-03-01
📅 2020-03-01 to 2020-03-31
📅 2020-03-31 to 2020-04-30
📅 2020-04-30 to 2020-05-30
📅 2020-05-30 to 2020-06-29
📅 2020-06-29 to 2020-07-29
📅 2020-07-29 to 2020-08-28
📅 2020-08-28 to 2020-09-27
📅 2020-09-27 to 2020-10-27
📅 2020-10-27 to 2020-11-26
📅 2020-11-26 to 2020-12-26
📅 2020-12-26 to 2021-01-25
📅 2021-01-25 to 2021-02-24
📅 2021-02-24 to 2021-03-26
📅 2021-03-26 to 2021-04-25
📅 2021-04-25 to 2021-05-25
📅 2021-05-25 to 2021-06-24
📅 2021-06-24 to 2021-07-24
📅 2021-07-24 to 2021-08-23
📅 2021-08-23 to 2021-09-22
📅 2021-09-22 to 2021-10-22
📅 2021-10-22 to 2021-11-21
📅 2021-11-21 to 2021-12-21
📅 2021-12-21 to 2022-01-20
📅 2022-01-20 to 2022-02-19
📅 2022-02-19 to 2022-03-21
📅 2022-03-21 to 2022-04-20
📅 2022-04-20 to 2022-05-20
📅 2022-05-20 to 2022-06-19
📅 2022-06-19 to 2022-07-19
📅 2022-07-19 to 2022-08-18
📅 2022-08-18 to 2022-09-17
📅 2022-09-17 to 2022-10-17
📅 2022-10-17 to 2022-11-16
📅 2022-11-16 to 2022-12-16
📅 2022-12-16 to 2023-01-15
📅

In [23]:
# 🌎 Set region to entire U.S. mainland
region = ee.Geometry.Rectangle([-125, 24, -66, 49])  # USA mainland

# 💡 Increase sample points for better data (~500 per month)
sample_points = ee.FeatureCollection.randomPoints(region, 1500)

# Set date range
start_date = datetime(2022, 1, 1)
end_date = datetime(2024, 12, 31)

# Prepare result container
all_rows = []

# Loop through each month in the date range
while start_date <= end_date:
    next_month = start_date + timedelta(days=30)
    s_str = start_date.strftime('%Y-%m-%d')
    e_str = next_month.strftime('%Y-%m-%d')
    print(f"📅 Processing: {s_str} to {e_str}")

    try:
        # 🔥 MODIS Burned Area (historical fires)
        burned = ee.ImageCollection("MODIS/061/MCD64A1") \
            .filterDate(s_str, e_str) \
            .select('BurnDate') \
            .mean() \
            .rename('BurnedArea')

        # 🌿 Sentinel-2 NDVI (vegetation health)
        ndvi = ee.ImageCollection("COPERNICUS/S2_SR") \
            .filterDate(s_str, e_str) \
            .filterBounds(region) \
            .map(lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI')) \
            .mean()

        # 🌡️ GLDAS Climate data
        climate = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H") \
            .filterDate(s_str, e_str) \
            .select(['Tair_f_inst', 'Wind_f_inst', 'Qair_f_inst']) \
            .mean() \
            .rename(['Temperature', 'WindSpeed', 'Humidity'])

        # 🏔️ Elevation
        elevation = ee.Image("USGS/SRTMGL1_003").select('elevation')

        # Combine all bands into one composite
        composite = ee.Image.cat([burned, ndvi, climate, elevation])

        # Sample the image at generated points
        sampled = composite.sampleRegions(
            collection=sample_points,
            scale=30,
            geometries=True
        )

        # Convert sample to Python list
        data = sampled.getInfo()
        for f in data['features']:
            p = f['properties']
            all_rows.append({
                'date': s_str,
                'NDVI': p.get('NDVI'),
                'Temperature': p.get('Temperature'),
                'Humidity': p.get('Humidity'),
                'WindSpeed': p.get('WindSpeed'),
                'Elevation': p.get('elevation'),
                'BurnedArea': p.get('BurnedArea', 0)
            })

    except Exception as e:
        print(f"⚠️ Skipped {s_str}: {e}")

    start_date = next_month

# Convert list to DataFrame
df = pd.DataFrame(all_rows)

# 🔥 Classify risk level based on BurnedArea
def classify_risk(val):
    if val is None:
        return 'Unknown'
    elif val > 150:
        return 'High'
    elif val > 50:
        return 'Medium'
    elif val > 0:
        return 'Low'
    else:
        return 'No Fire'

df['risk_level'] = df['BurnedArea'].fillna(0).apply(classify_risk)
df.drop(columns=['BurnedArea'], inplace=True)

# Save to CSV
df.to_csv("wildfire_data_2020_2024.csv", index=False)
print("✅ Dataset saved as wildfire_data_2020_2024.csv")

📅 Processing: 2022-01-01 to 2022-01-31
📅 Processing: 2022-01-31 to 2022-03-02
📅 Processing: 2022-03-02 to 2022-04-01
📅 Processing: 2022-04-01 to 2022-05-01
📅 Processing: 2022-05-01 to 2022-05-31
📅 Processing: 2022-05-31 to 2022-06-30
📅 Processing: 2022-06-30 to 2022-07-30
📅 Processing: 2022-07-30 to 2022-08-29
📅 Processing: 2022-08-29 to 2022-09-28
📅 Processing: 2022-09-28 to 2022-10-28
📅 Processing: 2022-10-28 to 2022-11-27
📅 Processing: 2022-11-27 to 2022-12-27
📅 Processing: 2022-12-27 to 2023-01-26
📅 Processing: 2023-01-26 to 2023-02-25
📅 Processing: 2023-02-25 to 2023-03-27
📅 Processing: 2023-03-27 to 2023-04-26
📅 Processing: 2023-04-26 to 2023-05-26
📅 Processing: 2023-05-26 to 2023-06-25
📅 Processing: 2023-06-25 to 2023-07-25
📅 Processing: 2023-07-25 to 2023-08-24
📅 Processing: 2023-08-24 to 2023-09-23
📅 Processing: 2023-09-23 to 2023-10-23
📅 Processing: 2023-10-23 to 2023-11-22
📅 Processing: 2023-11-22 to 2023-12-22
📅 Processing: 2023-12-22 to 2024-01-21
📅 Processing: 2024-01-21 

In [1]:
import pandas as pd

# Reload the uploaded dataset after environment reset
file_path = "/content/wildfire_data_2020_2024.csv"
df = pd.read_csv(file_path)

# Show structure, class distribution, and preview
df.info(), df['risk_level'].value_counts(), df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1082 entries, 0 to 1081
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         1082 non-null   object 
 1   NDVI         1082 non-null   float64
 2   Temperature  1082 non-null   float64
 3   Humidity     1082 non-null   float64
 4   WindSpeed    1082 non-null   float64
 5   Elevation    1082 non-null   int64  
 6   risk_level   1082 non-null   int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 59.3+ KB


(None,
 risk_level
 1    1082
 Name: count, dtype: int64,
          date      NDVI  Temperature  Humidity  WindSpeed  Elevation  \
 0  2022-01-31  0.161330   283.490265  0.005948   4.495435         44   
 1  2022-03-02  0.107536   265.759155  0.002246   3.570754        459   
 2  2022-03-02  0.084244   266.571350  0.002404   3.350337        348   
 3  2022-03-02  0.103986   283.623413  0.005708   3.209504        322   
 4  2022-03-02  0.046712   265.285980  0.002127   3.822837        540   
 
    risk_level  
 0           1  
 1           1  
 2           1  
 3           1  
 4           1  )

In [2]:
df['risk_level'].value_counts()

,count
risk_level,
1,1082


In [31]:
df.to_csv("wildfire_data_2020_2024.csv", index=False)

model training

In [3]:
import pandas as pd

# Load from Colab or upload again
df = pd.read_csv("/content/wildfire_data_2020_2024.csv")
df.head()


,date,NDVI,Temperature,Humidity,WindSpeed,Elevation,risk_level
0,2022-01-31,0.161330,283.490265,0.005948,4.495435,44,1
1,2022-03-02,0.107536,265.759155,0.002246,3.570754,459,1
2,2022-03-02,0.084244,266.571350,0.002404,3.350337,348,1
3,2022-03-02,0.103986,283.623413,0.005708,3.209504,322,1
4,2022-03-02,0.046712,265.285980,0.002127,3.822837,540,1


In [4]:
# Convert all fire classes (Low, Medium, High) to 1
df['risk_level'] = df['risk_level'].map({
    'No Fire': 0,
    'Low': 1,
    'Medium': 1,
    'High': 1
})


In [5]:
df['risk_level'].value_counts()


,count
risk_level,


In [6]:
df = pd.read_csv("/content/wildfire_data_2020_2024.csv")
print(df['risk_level'].unique())


[1]


In [7]:
df['risk_level'] = df['risk_level'].map({
    'No Fire': 0,
    'Low': 1,
    'Medium': 1,
    'High': 1
})


In [8]:
df['risk_level'] = df['risk_level'].apply(lambda x: 0 if x == 0 else 1)


In [9]:
print(df['risk_level'].value_counts())


risk_level
1    1082
Name: count, dtype: int64


In [35]:
print("Class counts:")
print(df['risk_level'].value_counts())


Class counts:
Series([], Name: count, dtype: int64)


In [37]:
print("Class counts:")
print(df['risk_level'].value_counts())


Class counts:
Series([], Name: count, dtype: int64)


In [29]:
print("Train class counts:", y_train.value_counts().to_dict())
print("Test class counts:", y_test.value_counts().to_dict())


Train class counts: {1: 865}
Test class counts: {1: 217}


In [1]:
import pandas as pd
df = pd.read_csv("/content/wildfire_data_2020_2024.csv")
print(df['risk_level'].unique())
print(df['risk_level'].value_counts())


['Low' 'No Fire' 'High' 'Medium']
risk_level
No Fire    1071
High          8
Low           2
Medium        1
Name: count, dtype: int64


In [2]:
df['risk_level'] = df['risk_level'].map({
    'No Fire': 0,
    'Low': 1,
    'Medium': 1,
    'High': 1
})


In [3]:
print(df['risk_level'].value_counts())


risk_level
0    1071
1      11
Name: count, dtype: int64


In [4]:
X = df.drop(['risk_level', 'date'], axis=1)
y = df['risk_level']


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)


In [6]:
from xgboost import XGBClassifier

model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [21:49:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [7]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['No Fire', 'Fire']))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

     No Fire       0.99      1.00      1.00       215
        Fire       0.00      0.00      0.00         2

    accuracy                           0.99       217
   macro avg       0.50      0.50      0.50       217
weighted avg       0.98      0.99      0.99       217

Confusion Matrix:
[[215   0]
 [  2   0]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8]:
from sklearn.utils import resample

# Separate classes
fire = df[df['risk_level'] == 1]
no_fire = df[df['risk_level'] == 0]

# Oversample fire
fire_upsampled = resample(fire, replace=True, n_samples=100, random_state=42)

# Combine
df_balanced = pd.concat([no_fire, fire_upsampled])
df_balanced = df_balanced.sample(frac=1, random_state=42)  # shuffle

# Redo split
X = df_balanced.drop(['risk_level', 'date'], axis=1)
y = df_balanced['risk_level']


In [9]:
from sklearn.utils import resample

# Separate classes
fire = df[df['risk_level'] == 1]
no_fire = df[df['risk_level'] == 0]

# Upsample fire class to 100 rows
fire_upsampled = resample(fire, replace=True, n_samples=100, random_state=42)

# Combine and shuffle
df_balanced = pd.concat([no_fire, fire_upsampled])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)


In [10]:
X = df_balanced.drop(['risk_level', 'date'], axis=1)
y = df_balanced['risk_level']


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)


In [12]:
from xgboost import XGBClassifier

model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [21:54:04] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [13]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['No Fire', 'Fire']))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

     No Fire       1.00      1.00      1.00       215
        Fire       0.95      1.00      0.98        20

    accuracy                           1.00       235
   macro avg       0.98      1.00      0.99       235
weighted avg       1.00      1.00      1.00       235

Confusion Matrix:
[[214   1]
 [  0  20]]


In [14]:
import joblib
joblib.dump(model, "wildfire_model.pkl")


['wildfire_model.pkl']

In [16]:
# //streamlit app

In [18]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.8 MB/s eta 0:00:00


In [19]:
import streamlit as st
import joblib

# Load model
model = joblib.load("/content/wildfire_model.pkl")

st.title("🔥 Wildfire Risk Prediction (Offline AI Model)")
st.markdown("Enter current environmental conditions to check fire risk.")

# Inputs
ndvi = st.slider("NDVI", -1.0, 1.0, 0.3)
temperature = st.number_input("Temperature (Kelvin)", value=300.0)
humidity = st.number_input("Humidity (kg/kg)", value=0.005)
wind = st.number_input("Wind Speed (m/s)", value=3.0)
elevation = st.number_input("Elevation (m)", value=500)

if st.button("🚀 Predict Risk"):
    prediction = model.predict([[ndvi, temperature, humidity, wind, elevation]])[0]
    label = "🔥 Fire Risk" if prediction == 1 else "✅ No Fire"
    st.success(f"Prediction: **{label}**")


2025-03-22 22:04:09.577 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-22 22:04:09.781 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-03-22 22:04:09.786 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-22 22:04:09.789 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-22 22:04:09.791 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-22 22:04:09.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-22 22:04:09.796 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-22 22:04:09.799 Thread 'MainThread': mi

In [2]:
!pip install streamlit-folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 9.2 MB/s eta 0:00:00


In [7]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 4s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸

In [13]:
!pip install pyngrok

In [19]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
added 22 packages in 3s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧

In [34]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.3 MB/s eta 0:00:00


In [35]:
!pip install pyngrok

In [36]:
!pip install streamlit_folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 5.3 MB/s eta 0:00:00


In [37]:
from pyngrok import ngrok

In [38]:
import ee
ee.Authenticate()
ee.Initialize(project='tidal-hack25tex-230')


In [16]:
!pip install streamlit-authenticator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 27.8 MB/s eta 0:00:00


In [85]:
# ✅ Must be first!
%%writefile app1.py
# ✅ Must be first!
import streamlit as st
import os
import time

st.set_page_config(page_title="Login", layout="centered")

# === Session State Auth Check ===
if "authenticated" not in st.session_state:
    st.session_state.authenticated = False

# === Styling ===
st.markdown("""
    <style>
      body {
            background-image: url('https://images.unsplash.com/photo-1572204097183-e1ab140342ed?q=80&w=2670&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D');
            background-size: cover;
            background-repeat: no-repeat;
            background-attachment: fixed;
            background-position: center;
        }
        .main-title {
            font-size: 36px;
            font-weight: bold;
            text-align: center;
            color: #10b981;
            margin-bottom: 1rem;
        }
        .subtext {
            text-align: center;
            color: #94a3b8;
        }
        .stApp {
            padding: 2rem;
            background-color: rgba(0, 0, 0, 0.5);
        }
        .login-box {
            background-color: #1e293b;
            padding: 2rem;
            border-radius: 1rem;
            max-width: 500px;
            margin: auto;
            box-shadow: 0px 0px 20px rgba(0,0,0,0.3);
          @media only screen and (max-width: 768px) {
            .main-title {
                font-size: 28px;
            }
            .report-box {
                font-size: 16px;
            }

        }
    </style>
""", unsafe_allow_html=True)

# === Login UI ===
st.markdown("<div class='main-title'>🔐 Login to Wildfire Dashboard</div>", unsafe_allow_html=True)
st.markdown("<div class='subtext'>Use your Google email to access wildfire predictions and weather features</div>", unsafe_allow_html=True)

with st.container():
    st.markdown("<div class='login-box'>", unsafe_allow_html=True)
    user_email = st.text_input("📧 Enter your Google Email")
    if user_email and not st.session_state.authenticated:
        st.success(f"✅ Logged in as {user_email}")
        st.session_state["authenticated"] = True
        st.session_state["user_email"] = user_email

        st.success("🔁 Redirecting to Wildfire Dashboard...")
        time.sleep(2)
        st.success("✅ Logged in! Go to the dashboard from the sidebar.")
    st.markdown("</div>", unsafe_allow_html=True)

# === Restrict all pages if not authenticated ===
if not st.session_state.authenticated:
    st.stop()

# === Show logout on all pages ===
def show_header():
    st.sidebar.title("👋 Welcome")
    st.sidebar.write(f"Logged in as **{st.session_state.get('user_email', 'Guest')}**")
    if st.sidebar.button("🚪 Logout"):
        st.session_state.authenticated = False
        st.session_state.user_email = None
        st.rerun()

# Call show_header() where needed in other pages



Overwriting app1.py


In [86]:

ngrok.set_auth_token('2uh3fIZrmGMThL8txxkFZ6mckbj_3WMbNENshZUPRY8XZt78b')

In [87]:
ngrok.kill()

In [88]:
!nohup streamlit run app1.py --server.port 8501 &
ngrok_tunnel = ngrok.connect(addr="8501", proto="http", bind_tls=True)
print("Tunnel URL:", ngrok_tunnel.public_url)


nohup: appending output to 'nohup.out'
Tunnel URL: https://b326-34-16-166-40.ngrok-free.app


In [89]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import streamlit as st

def send_email_manual(subject, body, recipient):
    sender_email = "merugusaicharan010601@gmail.com"  # Replace with your Gmail
    sender_password = "saicharan@v0"  # Use an App Password (not Gmail login!)

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.send_message(msg)
        st.success("📧 Email sent to " + recipient)
    except Exception as e:
        st.error("❌ Failed to send email: " + str(e))


In [58]:
send_email_manual(
    subject="🌦️ Your AI Weather Report",
    body="weather_report_text",
    recipient=st.session_state.get("merugusaicharan010601@gmail.com", "merugusaicharan010601@gmail.com")
)


2025-03-23 15:36:14.075 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-23 15:36:14.582 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-23 15:36:14.583 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [60]:
import smtplib
from email.mime.text import MIMEText

sender = "merugusaicharan010601@gmail.com"
password = "rcie sstk owkx dode"
recipient = "merugusaicharan010601@gmail.com"

msg = MIMEText("Hello from the wildfire app!")
msg['Subject'] = "Test Email"
msg['From'] = sender
msg['To'] = recipient

try:
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(sender, password)
    server.sendmail(sender, recipient, msg.as_string())
    server.quit()
    print("✅ Email sent")
except Exception as e:
    print("❌ Error:", e)


✅ Email sent
